# Steady State Solve

Example is based on Julian's Laplace example.

In [1]:
import petsc4py.PETSc as PETSc
import underworld3 as uw

In [2]:
options = PETSc.Options()

In [3]:
options.setValue("dm_plex_separate_marker", None)
options.setValue("temperature_petscspace_degree", 1)
options.setValue("sol_vec_view", None)
options.setValue("dm_view", None)
options.setValue("options_view",None)

In [4]:
user = {"y0": -0.6,
        "y1": 1.3,
        "k": 0.5,
        "h": 10,
        "T0": 4.0,
        "T1": 8.0,
        "simplex": True}

In [5]:
snes = PETSc.SNES().create(comm=PETSc.COMM_WORLD)

## Create Mesh

In [6]:
plex = PETSc.DMPlex().createBoxMesh(faces=(10,10), 
                                    lower=(-1.0, user["y0"]),
                                    upper=(1.0, user["y1"]),
                                    simplex=user["simplex"])

In [7]:
plex.distribute()

In [8]:
plex.setFromOptions()

In [9]:
plex.view()

In [10]:
part = plex.getPartitioner()
part.setFromOptions()
plex.distribute()

In [11]:
plex.localizeCoordinates()
plex.setFromOptions()

## Setup Discretisation

In [12]:
# Get Communicator from DM
comm = plex.comm
# Create Default FE
fe_temp = PETSc.FE().createDefault(dim=2,
                                   nc=1,
                                   isSimplex=user["simplex"],
                                   prefix="temperature_",
                                   qorder=1, comm=comm)
plex.setNumFields(1)
plex.setField(0, fe_temp)
plex.createDS()
prob = plex.getDS()

In [13]:
fe_temp.view()

### Setup Problems

In [14]:
from underworld3.poisson import PoissonSetup

In [15]:
PoissonSetup(plex, prob, user)

In [16]:
u = plex.createGlobalVector()

In [17]:
u.array[:] = 1.0

In [18]:
u.array

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [19]:
lu = plex.createLocalVector()

In [20]:
lu.array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [21]:
plex.insertBoundaryValues(True, lu, 0)

In [22]:
lu.array

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8.])

In [23]:
plex.localToGlobal(lu, u, addv=None)

In [24]:
u.array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [25]:
type(u)

petsc4py.PETSc.Vec

In [26]:
plex.globalToLocal(u, lu, addv=None)

In [27]:
lu.array

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8.])

In [31]:
fe_temp.destroy()

In [33]:
snes.setDM(plex)

In [34]:
#plex.createClosureIndex(None)

In [35]:
plex.setSNESLocalFEM()

In [36]:
#snes.setFromOptions()

In [37]:
import numpy as np
u[:] = np.ones_like(u.array) * 0.0

In [38]:
u.view()

In [39]:
u.setName("Initial Solution")
u.setName("Solution")

In [40]:
prob.view()

In [41]:
snes.solve(None, u)

In [42]:
u.view()

In [44]:
viewer = PETSc.Viewer().createHDF5("temperature.h5", "w")
viewer(u)
viewer(plex)
del viewer
from underworld3.petsc_gen_xdmf import generateXdmf
generateXdmf("temperature.h5")